In [3]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# Distances
import scipy.spatial.distance as distance
# KL
from scipy.stats import entropy
# Normalize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances
import h5py
import logging

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Adding variables
- NAICS: PC Match, Patents share inventor, Patent assignee share inventor
- PC: Patents share inventor, Patent assignee share inventor

### Common inventor at establishment (assignee-MSA)

1. Get patent-assignee-msaname
2. Get patent-inventors
3. Merge
4. Groupby assignee_id, msaname (establishment). Have to do this by grant year. Only get inventors in previous 10 years at establishment.

In [38]:
# # Get assignees and their locations (not inventors)
# asg = fastparquet.ParquetFile("RawData/Cleaned/patent_assignees_0628.parq").to_pandas(["assignee_id", "msaname", "patent", "type"])
# asg = asg.groupby(["patent", "assignee_id", "msaname"]).count().reset_index().sort_values("type", ascending=False).drop_duplicates("patent")
# asg = asg.drop("type",1)


# # Merge inventors to patents
# inv = fastparquet.ParquetFile("RawData/Cleaned/patent_inventors_0628.parq").to_pandas(["patent", "inventor_id"])
# inv = inv.merge(asg, how="left", on="patent")
# del(asg)
# print(len(inv))
# inv = inv.dropna(how="any")
# print(len(inv))

# # Merge grant year
# pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
# inv = inv.merge(pdf, how="left", on="patent")
# del(pdf)
# print(len(inv))

5017246
4438592
4438592


In [39]:
# Save inventor-assignee-patent data
# fastparquet.write("RawData/Cleaned/patent_inv_asg_0808.parq", inv, compression="GZIP")

In [9]:
%%time
inv = fastparquet.ParquetFile("RawData/Cleaned/patent_inv_asg_0808.parq").to_pandas()
inv_dict = {}
for gy in range(1976,2016):
    p2 = inv.loc[inv["gyear"].isin(range(gy-10,gy)), ["assignee_id", "msaname", "inventor_id"]].\
    groupby(["assignee_id", "msaname"])
    invd = {n+(gy,): g["inventor_id"].tolist() for n,g in p2}
    inv_dict.update(invd)
    del(p2)

CPU times: user 5min 12s, sys: 2.18 s, total: 5min 14s
Wall time: 5min 14s


### Get necessary stuff ready for other files

In [8]:
# Save necessary data
files = ["reg_naics_name_sim_tr_0726.parq", "reg_primclass_sim_tr_0726.parq"]
pathdir = "DataStore/2018-07-P2/Reg0726/"
for f in files:
    mdc = fastparquet.ParquetFile(pathdir+f).to_pandas(columns=['tp', 'op', 'tp_gyear', 'tp_inv_msa', 'op_inv_msa'])
    print(len(mdc))
    print(f)
    # Other patent grant year
    pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
    mdc["op_gyear"] = mdc["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))
    del(pdf)
    
    # Unique assignee for each patent
    asg = fastparquet.ParquetFile("RawData/Cleaned/patent_inv_asg_0808.parq").to_pandas(["patent", "assignee_id", "msaname"]).drop_duplicates()
    mdc = mdc.merge(asg.add_prefix("tp_"), how = "left", left_on = "tp", right_on = "tp_patent").drop("tp_patent",1)
    mdc = mdc.merge(asg.add_prefix("op_"), how = "left", left_on = "op", right_on = "op_patent").drop("op_patent",1)
    del(asg)
    
    # Save output
    if "naics_name" in f:
        fn = "naics_name_common_inv_0726.parq"
    else:
        fn = "primclass_common_inv_0726.parq"
    
    fastparquet.write(pathdir+fn, mdc, compression="GZIP")
    del(mdc)

1498184
reg_naics_name_sim_tr_0726.parq
1369833
reg_primclass_sim_tr_0726.parq


In [10]:
# Load new files and get common inventors
files = ["naics_name_common_inv_0726.parq", "primclass_common_inv_0726.parq"]
# files = ["naics_name_common_inv_0726.parq"]
pathdir = "DataStore/2018-07-P2/Reg0726/"

for f in files:
    mdc = fastparquet.ParquetFile(pathdir+f).to_pandas()
    print(f)
    print(len(mdc))
    %time num_common_inv = [len(set(inv_dict[tp]).intersection(inv_dict[op])) if (tp in inv_dict.keys()) & (op in inv_dict.keys())\
                      else np.nan for tp, op in zip(list(zip(mdc["tp_assignee_id"], mdc["tp_msaname"], mdc["tp_gyear"])),\
                                                    list(zip(mdc["op_assignee_id"], mdc["op_msaname"], mdc["op_gyear"])))]
    mdc["num_common_est_inv"] = num_common_inv
    del(num_common_inv)
    mdc["common_est_inv"] = np.nan
    mdc.loc[mdc["num_common_est_inv"] >= 1, "common_est_inv"] = True
    mdc.loc[mdc["num_common_est_inv"] == 0, "common_est_inv"] = False
    print(mdc["common_est_inv"].value_counts())
    fastparquet.write(pathdir+f, mdc, compression="GZIP")
    del(mdc)

naics_name_common_inv_0726.parq
1498184
CPU times: user 10min 29s, sys: 150 ms, total: 10min 29s
Wall time: 10min 28s
False    656592
True     158028
Name: common_est_inv, dtype: int64
primclass_common_inv_0726.parq
1369833
CPU times: user 10min 6s, sys: 143 ms, total: 10min 6s
Wall time: 10min 4s
False    585932
True     181347
Name: common_est_inv, dtype: int64


In [11]:
del(inv_dict)

### Add number of common inventors in each patent

In [12]:
inv = fastparquet.ParquetFile("RawData/Cleaned/patent_inventors_0628.parq").to_pandas(["patent", "inventor_id"])
%time inv = {n: g["inventor_id"].tolist() for n, g in inv.groupby("patent")}

CPU times: user 7min 10s, sys: 313 ms, total: 7min 10s
Wall time: 7min 9s


In [13]:
# Load new files and get common inventors
files = ["naics_name_common_inv_0726.parq", "primclass_common_inv_0726.parq"]
# files = ["naics_name_common_inv_0726.parq"]
pathdir = "DataStore/2018-07-P2/Reg0726/"

for f in files:
    mdc = fastparquet.ParquetFile(pathdir+f).to_pandas()
    print(f)
    print(len(mdc))
    %time num_common_inv = [len(set(inv[tp]).intersection(inv[op])) if (tp in inv.keys()) & (op in inv.keys())\
                      else np.nan for tp, op in zip(mdc["tp"], mdc["op"])]
    mdc["num_common_pat_inv"] = num_common_inv
    del(num_common_inv)
    mdc["common_pat_inv"] = np.nan
    mdc.loc[mdc["num_common_pat_inv"] >= 1, "common_pat_inv"] = True
    mdc.loc[mdc["num_common_pat_inv"] == 0, "common_pat_inv"] = False
    print(mdc["num_common_pat_inv"].value_counts())
    print(len(mdc))
    fastparquet.write(pathdir+f, mdc, compression="GZIP")
    del(mdc)

naics_name_common_inv_0726.parq
1498184
CPU times: user 4.25 s, sys: 11.1 ms, total: 4.26 s
Wall time: 4.23 s
0    1496604
1       1408
2        143
3         26
4          3
Name: num_common_pat_inv, dtype: int64
1498184
primclass_common_inv_0726.parq
1369833
CPU times: user 4.77 s, sys: 7.98 ms, total: 4.77 s
Wall time: 4.77 s
0    1365446
1       3779
2        470
3        105
4         20
5          6
8          3
6          3
7          1
Name: num_common_pat_inv, dtype: int64
1369833


### Add lawyers indicator

In [11]:
ldf = pd.read_table("RawData/PatentView/patent_lawyer.tsv")

def get_numeric(p):
    try:
        return int(p)
    except Exception:
        return np.nan
    
ldf["patent"] = ldf["patent_id"].apply(get_numeric)
ldf = ldf.loc[ldf["patent"].notnull(), ["patent", "lawyer_id"]]
ldf.to_csv("RawData/Cleaned/patent_lawyer.csv")

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Load new files and get common inventors
files = ["naics_name_common_inv_0726.parq", "primclass_common_inv_0726.parq"]
# files = ["naics_name_common_inv_0726.parq"]
pathdir = "DataStore/2018-07-P2/Reg0726/"
ldf = pd.read_csv("RawData/Cleaned/patent_lawyer.csv")
%time ldf = {n:g["lawyer_id"].tolist() for n,g in ldf[["patent", "lawyer_id"]].groupby("patent")}

for f in files:
    %time mdc = fastparquet.ParquetFile(pathdir+f).to_pandas()
    print(f)
    print(len(mdc))
    
    %time l_match = (set(ldf.get(tp, [])).intersection(set(ldf.get(op, []))) for tp, op in zip(mdc["tp"], mdc["op"]))
    %time mdc["lawyer_match"] = [len(i) for i in l_match]
    %time mdc["lawyer_match"] = mdc["lawyer_match"].apply(lambda x: True if x >= 1 else False)
    print(mdc["lawyer_match"].value_counts())
    print(len(mdc))
    fastparquet.write(pathdir+f, mdc, compression="GZIP")

### Add direct citation indicator

In [21]:
cit = dd.read_parquet("RawData/Cleaned/cit_0628.parq")

# Load new files and get common inventors
# files = ["naics_name_common_inv_0726.parq", "primclass_common_inv_0726.parq"]
files = ["primclass_common_inv_0726.parq"]
pathdir = "DataStore/2018-07-P2/Reg0726/"

for f in files:
    %time mdc = fastparquet.ParquetFile(pathdir+f).to_pandas()
    print(f)
    print(len(mdc))
    
#     # Direct citations
#     # Create all False
#     mdc["direct_cite"] = False
#     # Create zipped pairs
#     mdc["tp_op"] = list(zip(mdc["tp"], mdc["op"]))
#     # Citations
#     c2 = cit[cit["cited"].isin(mdc["tp"])].compute()
#     mdc.loc[mdc["tp_op"].isin(list(zip(c2["cited"], c2["citing"]))), "direct_cite"] = True
#     del(c2)
#     print(mdc["direct_cite"].value_counts())
#     # Drop pairs
#     mdc = mdc.drop("tp_op",1)
    
    # Number of common citations
    # Citations
    %time c2 = cit[cit["citing"].isin(mdc["tp"]) | cit["citing"].isin(mdc["op"])].compute()
    # Dictionary of citing: cited patents
    %time c2 = {n:g["cited"].tolist() for n,g in c2.groupby("citing")}
    # Get number of overlapping
    %time num_common_cites = (set(c2.get(tp, [])).intersection(set(c2.get(op, []))) for tp, op in zip(mdc["tp"], mdc["op"]))
    %time mdc["num_common_cites"] = [len(i) for i in num_common_cites]
    # At least one number of common cites
    mdc["common_cites_match"] = False
    mdc.loc[(mdc["num_common_cites"] >= 1), "common_cites_match"] = True
    print(mdc["num_common_cites"].value_counts()[:20])
    print(mdc["common_cites_match"].value_counts())
    print(len(mdc))
    print(mdc.columns)
    fastparquet.write(pathdir+f, mdc, compression="GZIP")

CPU times: user 2.8 s, sys: 336 ms, total: 3.13 s
Wall time: 3.07 s
primclass_common_inv_0726.parq
1369833
CPU times: user 1min 6s, sys: 10.1 s, total: 1min 16s
Wall time: 28 s
CPU times: user 2min 44s, sys: 4.22 s, total: 2min 48s
Wall time: 2min 53s
CPU times: user 40.6 ms, sys: 26.1 ms, total: 66.7 ms
Wall time: 73.2 ms
CPU times: user 5.7 s, sys: 0 ns, total: 5.7 s
Wall time: 6.27 s
0     1351856
1       11156
2        2681
3        1207
4         644
5         413
6         286
7         220
8         160
9         125
11         96
10         86
12         63
13         44
15         43
14         42
17         39
19         36
18         35
20         30
Name: num_common_cites, dtype: int64
False    1351856
True       17977
Name: common_cites_match, dtype: int64
1369833
Index(['tp', 'op', 'tp_gyear', 'tp_inv_msa', 'op_inv_msa', 'op_gyear',
       'tp_assignee_id', 'tp_msaname', 'op_assignee_id', 'op_msaname',
       'num_common_est_inv', 'common_est_inv', 'num_common_pat_inv',
 

_______________

Finish here

In [86]:
# Add indicator term

# Load new files and get common inventors
files = ["naics_name_common_inv_0726.parq", "primclass_common_inv_0726.parq"]
# files = ["naics_name_common_inv_0726.parq"]
pathdir = "DataStore/2018-07-P2/Reg0726/"

for f in files:
    mdc = fastparquet.ParquetFile(pathdir+f).to_pandas()
    mdc["common_est_inv"] = np.nan
    mdc.loc[mdc["num_common_est_inv"] >= 1, "common_est_inv"] = True
    mdc.loc[mdc["num_common_est_inv"] == 0, "common_est_inv"] = False
    
    mdc["common_pat_inv"] = np.nan
    mdc.loc[mdc["num_common_pat_inv"] >= 1, "common_pat_inv"] = True
    mdc.loc[mdc["num_common_pat_inv"] == 0, "common_pat_inv"] = False
    fastparquet.write(pathdir+f, mdc, compression="GZIP")
    del(mdc)

In [3]:
# Load new files and get common inventors
# files = ["naics_name_common_inv_0726.parq", "primclass_common_inv_0726.parq"]
files = ["naics_name_common_inv_0726.parq"]
files = ["primclass_common_inv_0726.parq"]
pathdir = "DataStore/2018-07-P2/Reg0726/"

for f in files:
    mdc = fastparquet.ParquetFile(pathdir+f).to_pandas()

In [4]:
mdc["common_est_inv"].value_counts()

0.0    656592
1.0    158028
Name: common_est_inv, dtype: int64

In [5]:
mdc["common_pat_inv"].value_counts()

False    1496604
True        1580
Name: common_pat_inv, dtype: int64